# TiledMMA矩阵乘法 - 实践篇

本notebook通过概念讲解和代码分析帮助你理解cute的TiledMMA抽象。

**学习目标：**
- 理解Tensor Core和MMA操作的基本概念
- 分析FlashAttention中的TiledMMA使用
- 理解partition操作的含义


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 1. Tensor Core MMA操作概念


In [ ]:
def visualize_mma_atom():
    """可视化单个MMA Atom操作"""
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    # MMA 16x8x16: A(16x16) @ B(16x8) = C(16x8)
    
    # A矩阵
    ax = axes[0]
    A = np.random.rand(16, 16)
    ax.imshow(A, cmap='Blues')
    ax.set_title('A矩阵\n16×16 (FP16)')
    ax.set_xlabel('K=16')
    ax.set_ylabel('M=16')
    
    # B矩阵
    ax = axes[1]
    B = np.random.rand(16, 8)
    ax.imshow(B, cmap='Greens')
    ax.set_title('B矩阵\n16×8 (FP16)')
    ax.set_xlabel('N=8')
    ax.set_ylabel('K=16')
    
    # 乘法符号
    ax = axes[2]
    ax.text(0.5, 0.5, '×', fontsize=60, ha='center', va='center')
    ax.axis('off')
    ax.set_title('矩阵乘法')
    
    # C矩阵
    ax = axes[3]
    C = np.random.rand(16, 8)
    ax.imshow(C, cmap='Oranges')
    ax.set_title('C矩阵\n16×8 (FP32)')
    ax.set_xlabel('N=8')
    ax.set_ylabel('M=16')
    
    plt.suptitle('SM80_16x8x16 MMA Atom: D = A × B + C', fontsize=14)
    plt.tight_layout()
    plt.show()

visualize_mma_atom()

print("MMA Atom规格: SM80_16x8x16_F32F16F16F32_TN")
print("- M=16: 输出矩阵的行数")
print("- N=8:  输出矩阵的列数")
print("- K=16: 累加的维度")
print("- 输入精度: FP16")
print("- 输出精度: FP32")
print("- TN: A转置, B不转置")


## 2. TiledMMA: 多个Atom组合


In [ ]:
def visualize_tiled_mma():
    """可视化TiledMMA如何组合多个Atom"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 左图: 单个Atom
    ax = axes[0]
    atom = plt.Rectangle((0, 0), 8, 16, fill=True, facecolor='lightblue', edgecolor='blue', linewidth=2)
    ax.add_patch(atom)
    ax.set_xlim(-1, 10)
    ax.set_ylim(-1, 18)
    ax.set_aspect('equal')
    ax.set_title('单个MMA Atom\n16×8 输出')
    ax.set_xlabel('N')
    ax.set_ylabel('M')
    ax.text(4, 8, 'Atom\n16×8', ha='center', va='center', fontsize=12)
    
    # 右图: TiledMMA (2x2 Atoms)
    ax = axes[1]
    colors = ['lightcoral', 'lightgreen', 'lightyellow', 'lightblue']
    labels = ['Atom(0,0)', 'Atom(0,1)', 'Atom(1,0)', 'Atom(1,1)']
    
    for i in range(2):
        for j in range(2):
            rect = plt.Rectangle((j*8, (1-i)*16), 8, 16, 
                                 fill=True, facecolor=colors[i*2+j], 
                                 edgecolor='black', linewidth=2)
            ax.add_patch(rect)
            ax.text(j*8+4, (1-i)*16+8, labels[i*2+j], ha='center', va='center', fontsize=10)
    
    ax.set_xlim(-1, 18)
    ax.set_ylim(-1, 34)
    ax.set_aspect('equal')
    ax.set_title('TiledMMA (AtomLayout=2×2)\n32×16 输出')
    ax.set_xlabel('N = 8×2 = 16')
    ax.set_ylabel('M = 16×2 = 32')
    
    plt.suptitle('TiledMMA组合多个MMA Atom覆盖更大矩阵', fontsize=14)
    plt.tight_layout()
    plt.show()

visualize_tiled_mma()

print("TiledMMA配置示例:")
print("- MMA Atom: SM80_16x8x16")
print("- AtomLayout: Shape<_2, _2, _1>  (2×2个Atom)")
print("- 输出矩阵大小: M=16×2=32, N=8×2=16")


## 3. 线程到数据的映射


In [ ]:
def visualize_thread_mapping():
    """可视化线程如何映射到MMA操作"""
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # 32个线程的简化映射 (16x8 输出矩阵)
    # 每个线程持有多个元素
    
    M, N = 16, 8
    thread_map = np.zeros((M, N), dtype=int)
    
    # 简化的线程映射模式
    for i in range(M):
        for j in range(N):
            # 这是一个简化模型，实际映射更复杂
            thread_map[i, j] = (i % 4) * 8 + j
    
    im = ax.imshow(thread_map, cmap='tab20', vmin=0, vmax=31)
    ax.set_title('MMA Atom中的线程映射 (简化示意)\\n颜色=线程ID', fontsize=12)
    ax.set_xlabel('N (列)')
    ax.set_ylabel('M (行)')
    
    for i in range(M):
        for j in range(N):
            ax.text(j, i, f'T{thread_map[i,j]}', ha='center', va='center', 
                   fontsize=7, color='white')
    
    plt.colorbar(im, ax=ax, label='线程ID')
    plt.tight_layout()
    plt.show()
    
    print("关键概念:")
    print("- 一个Warp (32线程) 协作完成一个MMA Atom")
    print("- 每个线程持有输出矩阵的多个元素 (Fragment)")
    print("- partition_C 返回当前线程负责的元素")

visualize_thread_mapping()


## 4. FlashAttention中的TiledMMA代码分析


In [ ]:
flashattention_tiledmma_code = '''
// ============================================================
// FlashAttention中的TiledMMA使用模式
// 来自 hopper/utils.h 和 mainloop_*.hpp
// ============================================================

// 1. TiledMMA类型定义
using TiledMmaSdP = TiledMMA<
    MMA_Atom<SM80_16x8x16_F32F16F16F32_TN>,
    Layout<Shape<_2, _2, _1>>,   // 2×2 Atoms
    Tile<_32, _32, _16>          // 输出Tile大小
>;

// 2. 在kernel中使用
__device__ void compute_attention() {
    TiledMmaSdP tiled_mma_SdP;
    
    // 获取当前线程的视图
    auto thr_mma = tiled_mma_SdP.get_thread_slice(threadIdx.x);
    
    // 创建累加器Fragment
    Tensor acc_s = partition_fragment_C(tiled_mma_SdP, 
                                        Shape<Int<kBlockM>, Int<kBlockN>>{});
    clear(acc_s);
    
    // 分区输入矩阵
    Tensor tSrQ = thr_mma.partition_A(sQ);  // Q的分区
    Tensor tSrK = thr_mma.partition_B(sK);  // K的分区
    
    // 执行矩阵乘法: S = Q @ K^T
    cute::gemm(tiled_mma_SdP, tSrQ, tSrK, acc_s);
    
    // acc_s 现在包含 S 的结果
}

// 3. gemm函数的封装 (utils.h)
template <bool zero_init=true, typename TiledMma, typename Tensor0, 
          typename Tensor1, typename Tensor2>
CUTLASS_DEVICE void gemm(TiledMma& tiled_mma, 
                         Tensor0 const& tCrA, 
                         Tensor1 const& tCrB, 
                         Tensor2& tCrC) {
    // K维度的循环
    constexpr int MMA_K = decltype(size<2>(tCrA))::value;
    
    #pragma unroll
    for (int k = 0; k < MMA_K; ++k) {
        cute::gemm(tiled_mma, tCrA(_, _, k), tCrB(_, _, k), tCrC);
    }
}
'''

print(flashattention_tiledmma_code)


## 5. 模拟partition操作


In [ ]:
def simulate_partition():
    """模拟partition操作的概念"""
    
    print("partition操作概念:")
    print("=" * 60)
    
    # 假设的矩阵大小
    M, N, K = 128, 64, 64
    num_threads = 128
    threads_per_warp = 32
    num_warps = num_threads // threads_per_warp
    
    print(f"矩阵大小: A={M}×{K}, B={K}×{N}, C={M}×{N}")
    print(f"线程数: {num_threads} ({num_warps} warps)")
    print()
    
    # TiledMMA配置
    mma_m, mma_n, mma_k = 16, 8, 16
    atom_layout_m, atom_layout_n = 2, 2
    
    tile_m = mma_m * atom_layout_m  # 32
    tile_n = mma_n * atom_layout_n  # 16
    
    print(f"MMA Atom大小: {mma_m}×{mma_n}×{mma_k}")
    print(f"Atom Layout: {atom_layout_m}×{atom_layout_n}")
    print(f"每个TiledMMA输出: {tile_m}×{tile_n}")
    print()
    
    # 计算需要多少个TiledMMA
    num_tiles_m = M // tile_m
    num_tiles_n = N // tile_n
    
    print(f"需要的TiledMMA数量: {num_tiles_m}×{num_tiles_n} = {num_tiles_m * num_tiles_n}")
    print()
    
    # partition后每个线程的数据量
    elements_per_atom = (mma_m * mma_n) // threads_per_warp
    elements_per_thread = elements_per_atom * atom_layout_m * atom_layout_n
    
    print(f"partition_C 结果:")
    print(f"  每个线程在一个Atom中: {elements_per_atom} 元素")
    print(f"  每个线程在TiledMMA中: {elements_per_thread} 元素")
    print(f"  Fragment形状类似: ((4), (2), (2)) for 2×2 Atoms")

simulate_partition()


## 总结

通过本notebook，你应该理解了：

1. **Tensor Core MMA**: 硬件级别的矩阵乘累加操作，如16×8×16
2. **MMA Atom**: 单个Tensor Core指令的抽象
3. **TiledMMA**: 多个Atom的组合，覆盖更大的矩阵
4. **partition**: 将矩阵分配给线程，返回Fragment
5. **cute::gemm**: 执行矩阵乘法的高层接口

## 关键代码模式

```cpp
// 1. 定义TiledMMA
TiledMma tiled_mma = make_tiled_mma(...);

// 2. 获取线程视图
auto thr_mma = tiled_mma.get_thread_slice(threadIdx.x);

// 3. 分区
Tensor tCrA = thr_mma.partition_A(sA);
Tensor tCrB = thr_mma.partition_B(sB);

// 4. 执行GEMM
cute::gemm(tiled_mma, tCrA, tCrB, acc);
```

## 下一步

在下一节"TiledCopy内存拷贝"中，我们将学习如何高效地在内存层次间传输数据。
